In [218]:
import numpy as np
import pandas as pd
import load_dataset as ld
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import multivariate_normal as mvn

## First load our dataset

In [219]:
exoplanet_path = '../published_output/exoplanet.eu_catalog_15April.csv'
solar_path = '../published_output/solar_system_planets_catalog.csv'
features_name = ['mass', 'semi_major_axis','eccentricity', 'star_metallicity',
                'star_radius', 'star_teff','star_mass', 'radius']

dataset = ld.load_dataset(exoplanet_path, solar_path, features_name)

In [220]:
features_name[:-1]

['mass',
 'semi_major_axis',
 'eccentricity',
 'star_metallicity',
 'star_radius',
 'star_teff',
 'star_mass']

In [221]:
dataset_exo = dataset[:-8]
dataset_solar = dataset[-8:]

In [222]:
features = dataset_exo.drop('radius', axis=1).copy()   # mass, teq, etc
labels = dataset_exo[['radius']].copy()   # radius

In [223]:
# features_1 = dataset_exo.iloc[:, :-1]   # mass, teq, etc
# labels_1 = dataset_exo.iloc[:, -1]      # radius

In [224]:
# random state is random seed here
# test size is percentage of test dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels['radius'], test_size=0.25, random_state=0)

In [225]:
# X_train, X_test, y_train, y_test = train_test_split(features_1, labels_1, test_size=0.25, random_state=0)

In [226]:
# TODO: PHYS, why they manually change the data ??
# TODO: Read paper to see if they mention it ??
X_test = X_test.drop(['HATS-12 b'])
y_test = y_test.drop(labels=['HATS-12 b'])
print('\nHATS-12 b removes from test set\n')

# Remove K2-95 b from the training set
X_train = X_train.drop(['K2-95 b'])
y_train = y_train.drop(labels=['K2-95 b'])
print('\nK2-95 b removes from training set\n')

# Remove Kepler-11 g from the training set
X_train = X_train.drop(['Kepler-11 g'])
y_train = y_train.drop(labels=['Kepler-11 g'])
print('\nKepler-11 g removes from training set\n')


HATS-12 b removes from test set


K2-95 b removes from training set


Kepler-11 g removes from training set


In [227]:
train_test_values = [X_train.values, X_test.values, y_train.values, y_test.values]
train_test_sets = [X_train, X_test, y_train, y_test]

In [228]:
# n_estimators: n of trees
# max_depth
# max features: the max value of features select for each node
# min_samples_split: if split number is smaller than this value, don't split a node
# TODO: min_samples_split, what np.arange(4, 5) mean??? dont make me laugh plz
rf = GridSearchCV(RandomForestRegressor(),
                          param_grid={'n_estimators': np.arange(80, 200),
                                      'max_depth': np.arange(4, 10),
                                      'max_features': np.arange(3, 6),
                                      'min_samples_split': np.arange(4, 5)},
                          cv=3, verbose=1, n_jobs=-1)
rf.fit(X_train.values, y_train.values.ravel())

Fitting 3 folds for each of 2160 candidates, totalling 6480 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': array([4, 5, 6, 7, 8, 9]),
                         'max_features': array([3, 4, 5]),
                         'min_samples_split': array([4]),
                         'n_estimators': array([ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196,
       197, 198, 199])},
             verbose=1)

In [229]:
regr = RandomForestRegressor(n_estimators=rf.best_params_['n_estimators'],
                             max_depth=rf.best_params_['max_depth'],
                             max_features=rf.best_params_['max_features'],
                             min_samples_split=rf.best_params_['min_samples_split'],
                             random_state=0, oob_score=True)
regr.fit(X_train.values, y_train.values.ravel())

RandomForestRegressor(max_depth=9, max_features=5, min_samples_split=4,
                      n_estimators=151, oob_score=True, random_state=0)

In [230]:
train_prediction = regr.predict(X_train.values)

In [231]:
# after we have the model, we can calculate the rmse of training and testing data
train_prediction = regr.predict(X_train.values)
train_rmse = np.sqrt(np.mean((train_prediction - y_train.values)**2))
test_prediction = regr.predict(X_test.values)
test_rmse = np.sqrt(np.mean((test_prediction - y_test.values)**2))
print(train_rmse, test_rmse)

1.0024773400278228 1.8168347537463303


In [232]:
X_train

,mass,semi_major_axis,star_luminosity,temp_eq,eccentricity,star_metallicity,star_radius,star_teff,star_mass
HAT-P-2 b,2777.820274,0.06740,3.603693,1537.034936,0.5171,0.04,1.540,6414.0,1.340
Kepler-91 b,232.014737,0.07200,15.272749,2053.457463,0.0660,0.11,6.300,4550.0,1.310
WASP-133 b,368.680952,0.03450,1.965237,1782.263273,0.1700,0.29,1.440,5700.0,1.160
WASP-16 b,271.743288,0.04210,0.762333,1268.617802,0.0000,0.01,0.946,5550.0,1.022
WASP-94 A b,143.658440,0.05500,2.406628,1479.471390,0.0000,0.26,1.360,6170.0,1.290
...,...,...,...,...,...,...,...,...,...
Kepler-9 b,43.510709,0.14300,1.164575,765.636501,0.0626,0.17,1.100,5722.0,1.000
HD 2685 b,375.037520,0.05680,4.734574,1724.176538,0.0000,0.02,1.570,6801.0,1.440
HATS-20 b,86.767155,0.04619,0.610133,1187.504126,0.5000,0.03,0.892,5406.0,0.910
GJ 9827 d,3.813941,0.06270,0.124724,661.134143,0.0000,-0.28,0.651,4255.0,0.659


In [233]:
# TODO, they calculate R2 score, what is that
# TODO: why they calculate the abs error but didn't give the relative error ??

## Calculate error bar

In [234]:
# after we have the model
# TODO: the feature is diff with original code
features_error = ['mass', 'semi_major_axis','eccentricity', 'star_radius', 'star_teff','star_mass', 'radius']
# dataset_error = ld.load_dataset_error(exoplanet_path, features_error, solar_path, solar=True)
dataset_error = ld.load_dataset_error(exoplanet_path, solar_path, features_name, solar=True)

                     mass  mass_error_min  mass_error_max  semi_major_axis  \
# name                                                                       
51 Peg b       149.379351       22.247988        9.534852         0.052000   
55 Cnc e         8.590902        0.429068        0.429068         0.015439   
BD+20 594 b     16.304597        6.038740        6.038740         0.241000   
BD-10 3166 b   146.201067       92.805895       83.271043         0.046000   
CoRoT-1 b      327.363259       38.139409       38.139409         0.025400   
...                   ...             ...             ...              ...   
XO-4 b         513.610705       31.782841       32.100669         0.054850   
XO-5 b         342.301194       11.759651       11.759651         0.048700   
XO-6 b         603.873973      158.914203      158.914203         0.081500   
kappa And b   4131.769286      635.656813     3813.940879       100.000000   
tau Boo b     1856.117895      174.805624      314.650123       

In [235]:
X_train, X_test, y_train, y_test = train_test_sets
# Cross matching the Test set with the dataset with errors
# to compute error bars for the exoplanets which have input errors
dataset_errors = dataset_error.loc[X_test.index.values.tolist()]

In [236]:
dataset_errors = dataset_errors.dropna(axis=0, how='any')

In [237]:
# Matrix with all the errors on the different features
features_errors = dataset_errors.iloc[:, :-2].values
# Radius vector
radii_test = dataset_errors.iloc[:, -2].values
# Error on the radius vector
radii_test_input_error = dataset_errors.iloc[:, -1].values
radii_test_output_error = np.zeros([len(radii_test_input_error)])

In [238]:
column_names = X_train.columns.tolist()
for i in range(125):
    try:
        normal_sample = mvn(features_errors[i, ::2], np.diag(features_errors[i, 1::2])).rvs(1000)
        sample_prediction_std = regr.predict(normal_sample).std()
        radii_test_output_error[i] = sample_prediction_std
    except ValueError:
        print(i)
        # This block executes if a ValueError is raised in the try block
        print(f"A ValueError occurred at {i}th column, but the script will continue.")

47
A ValueError occurred at 47th column, but the script will continue.
